In [6]:
import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import os
import kagglehub
from sklearn.model_selection import train_test_split

load_dotenv()

True

In [4]:
directory_path = kagglehub.dataset_download(
    "tanishqdublish/text-classification-documentation"
)
csv_file_path = os.path.join(directory_path, "df_file.csv")
df = pd.read_csv(csv_file_path)
df.head

<bound method NDFrame.head of                                                    Text  Label
0     Budget to set scene for election\n \n Gordon B...      0
1     Army chiefs in regiments decision\n \n Militar...      0
2     Howard denies split over ID cards\n \n Michael...      0
3     Observers to monitor UK election\n \n Minister...      0
4     Kilroy names election seat target\n \n Ex-chat...      0
...                                                 ...    ...
2220  India opens skies to competition\n \n India wi...      4
2221  Yukos bankruptcy 'not US matter'\n \n Russian ...      4
2222  Survey confirms property slowdown\n \n Governm...      4
2223  High fuel prices hit BA's profits\n \n British...      4
2224  US trade gap hits record in 2004\n \n The gap ...      4

[2225 rows x 2 columns]>

In [5]:
df.drop_duplicates(subset=["Text"], inplace=True)

In [11]:
df_700, temp = train_test_split(df, test_size=0.67, stratify=df["Label"])
len(df_700)

701

In [17]:
openai.api_key = os.getenv("OPENAI_API_KEY")


def translate_text(text):
    prompt = f"Translate the following English text into Ukrainian:\n\n{text}"

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2000,
    )

    return response.choices[0].message.content

In [19]:
tqdm.pandas()
df_700["translated_text"] = df_700["Text"].progress_apply(translate_text)


print("Translation completed and saved!")

100%|██████████| 701/701 [3:43:58<00:00, 19.17s/it]  

Translation completed and saved!


In [22]:
df_700["Label"] = df_700["Label"].replace(
    {
        "Politics": "Політика",
        "Sport": "Спорт",
        "Technology": "Технології",
        "Entertainment": "Розваги",
        "Business": "Бізнес",
    }
)

df_700 = df_700[["Label", "translated_text"]].rename(
    columns={"translated_text": "text"}
)

In [23]:
df_700.head

<bound method NDFrame.head of       Label                                               text
131       0  Доповідь про напади на витрати на оборону\n\nМ...
2157      4  Бразилія зменшує значення порятунку Varig\n\nУ...
219       0  Джемісон попереджає великих біготів\n\nМіністр...
682       1  Чарвіс, ймовірно, не зможе відновитися до матч...
2166      4  Азійські акції не піддаються постземлетрусному...
...     ...                                                ...
1500      3  Гімн благодійності США повторно випущено\n\nWe...
784       1  Венгер підтримує Альмунію\n\nАрсен Венгер пооб...
1977      4  Unilever зазнає змін через падіння прибутку\n\...
511       1  Тигри обережні щодо "ризику" з Фарреллом\n\nЛе...
112       0  Ліберальні демократи націлюються на покупців в...

[701 rows x 2 columns]>

In [24]:
df_700.to_csv("ukr_dataset.csv", index=False)